In [ ]:
import pandas as pd
import ast

In [ ]:
path="/srv/chawak/planning-with-llms/results/rl/training/"
#date="17_06/"
# date='debug-2006/'
date="09_07/"
df= pd.read_csv(f'{path}{date}metrics.csv')
#df=pd.read_csv(f"{path}{inference_dir}")
df=df.drop(columns=['Unnamed: 0'])

In [ ]:
df['Terminate']=df['Terminate'].apply(ast.literal_eval)

In [ ]:
def terminate_per_problem(lst, chunk_size):
    return [sum(lst[i:i + chunk_size]) for i in range(0, len(lst), chunk_size)]

In [ ]:
import yaml
with open("/srv/chawak/planning-with-llms/src/rl/config.yaml", "r") as f:
    cfg=yaml.safe_load(f)
group_size=cfg['training']['num_generations']

In [ ]:
df['Terminate_chunks'] = df['Terminate'].apply(lambda x: terminate_per_problem(x, group_size))

In [ ]:
df['Terminate_chunks']

In [ ]:
# transposed

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Data
metrics = ['reward/format_reward', 'reward/plan_reward', 'reward/bonus_reward', '# Terminate']
epochs = np.arange(len(df))
bar_width = 0.2

plt.figure(figsize=(12, 6))

for i, metric in enumerate(metrics):
    bar_positions = epochs + i * bar_width
    values = df[metric]
    
    # Plot bars
    bars = plt.bar(bar_positions, values, width=bar_width, label=metric)
    
    # Add value labels on top
    for bar in bars:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2, height + 0.3, f'{height:.1f}', 
                 ha='center', va='bottom', fontsize=8)

# X-axis setup
plt.xticks(epochs + bar_width * (len(metrics)-1)/2, [f'Epoch {i}' for i in epochs])
plt.xlabel('Epoch')
plt.ylabel('Value')
plt.title('Epoch-wise rewards and #plans that reach goal')
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Data
metrics = ['reward/format_reward', 'reward/plan_reward', 'reward/bonus_reward', '# Terminate']
epochs = np.arange(len(df))

plt.figure(figsize=(12, 6))

# Plot a separate line for each metric
for metric in metrics:
    values = df[metric]
    plt.plot(epochs, values, marker='o', label=metric)

    # Add value labels at each point
    for x, y in zip(epochs, values):
        plt.text(x, y + 0.2, f'{y:.1f}', ha='center', va='bottom', fontsize=8)

    #moving-average for plan-reward
    if metric=="reward/plan_reward":
        rolling_values=df[metric].rolling(window=3,min_periods=1).mean()
        plt.plot(epochs,rolling_values,linestyle='--',linewidth=2, label= 'reward/plan_reward(rolling avg of 3)')

# X-axis setup
plt.xticks(epochs, [f'Epoch {i}' for i in epochs])
plt.xlabel('Epoch')
plt.ylabel('Value')
plt.title('Epoch-wise rewards and #plans that reach goal')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


scrap for fun

In [ ]:
import regex as re

In [ ]:
gibberish=r"(.*)"
plan=r"\[PLAN\](.*?)\[PLAN END\]"
think=r"<think>(.*?)<\/think>"

In [ ]:
#case-1: no gibberish, no plan, yes think 
pattern1=f"{think}"
compiled1=re.compile(pattern1,re.DOTALL)

#case-2: no gibberish, yes plan, no think
pattern2=f"{plan}"
compiled2=re.compile(pattern2,re.DOTALL)

#case-3: no gibberish, yes plan, yes think
pattern3=f"{think}\s*{plan}"
compiled3=re.compile(pattern3,re.DOTALL)

#case-6: yes gibberish, yes plan, yes think
pattern6=f"{think}{gibberish}{plan}"
compiled6=re.compile(pattern6,re.DOTALL)


In [ ]:
def reward_func(response):
    
    response=response.strip()
    score=0

    #ONLY think tag
    fullmatchthink=False
    if compiled1.fullmatch(response):
        score=7
        fullmatchthink=True
    if compiled1.search(response) and not fullmatchthink:
        score=5
    
    #ONLY plan tag
    fullmatchplan=False
    if compiled2.fullmatch(response):
        score=3
        fullmatchplan=True
    if compiled2.search(response) and not fullmatchplan:
        score=2
    
    #COMBO of both tags
    fullmatchcombo=False
    if compiled3.fullmatch(response):
        score=20
        fullmatchcombo=True
    if compiled6.search(response) and not fullmatchcombo:
        score=15

    return score

In [ ]:
response='''<think>
\nFirst unstack the green block from on top of the red block.\n
Then put down the green block.\nThen pick up the red block.\nThen stack the red block on top of the green block.
\nThen pick up the pink block.\nThen stack the pink block on top of the red block.\n
</think>
\n[PLAN]
\nunstack the green block from on top of the red block\nput down the green block
\npick up the red block\nstack the red block on top of the green block\npick up the pink block
\nstack the pink block on top of the red block\n
[PLAN END]\n'''

In [ ]:
response='''<think>
\nFirst unstack the green block from on top of the red block.\n
Then put down the green block.\nThen pick up the red block.\nThen stack the red block on top of the green block.
\nThen pick up the pink block.\nThen stack the pink block on top of the red block.\n
</think>
\n[PLAN]
\nunstack the green block from on top of the red block\nput down the green block
\npick up the red block\nstack the red block on top of the green block\npick up the pink block
\nstack the pink block on top of the red block\n
[PLAN END]\n
'''

In [ ]:
reward_func(response)

In [ ]:
path="/srv/chawak/planning-with-llms/src/logs/"
#date="17_06/"
date='July/GRPO-train-1107.log'
log_file = f"{path}{date}"

In [ ]:
import re
import matplotlib.pyplot as plt

log_path = log_file  # or use your variable
target_pid = "3457368"
# target_pid="3131681"

pid_usage = []
timestamps = []

with open(log_path, "r") as f:
    lines = f.readlines()

i = 0
step = 0
while i < len(lines):
    if "NVIDIA-SMI" in lines[i]:
        j = i
        usage_this_block = 0
        in_process_section = False

        while j < len(lines):
            line = lines[j].strip()

            if line.startswith("| Processes:"):
                in_process_section = True

            elif in_process_section:
                # Look for a line containing the target PID
                if re.search(rf"\|\s+\d+\s+N/A\s+N/A\s+{target_pid}\s+", line):
                    match = re.search(r"(\d+)MiB", line)
                    if match:
                        mem = int(match.group(1))
                        usage_this_block += mem
                        print(f"Step {step}: found {mem} MiB on this GPU")

                # End of the process table
                if line.startswith("+-----------------------------------------------------------------------------------------+"):
                    break

            j += 1

        if usage_this_block > 0:
            pid_usage.append(usage_this_block)
            timestamps.append(step)
            step += 1

        i = j
    else:
        i += 1

print(f"Found {len(pid_usage)} data points for PID {target_pid}")

# Plot
plt.figure(figsize=(10, 6))
plt.plot(timestamps, pid_usage, marker='o', label=f'PID {target_pid}')
plt.xlabel("nvidia-smi snapshot index")
plt.ylabel("Memory Used (MiB)")
plt.title(f"GPU Memory Usage Over Time for PID {target_pid}")
plt.grid(True)
# plt.legend()
plt.tight_layout()
for x in range(0, len(timestamps)+1, 15):
    plt.axvline(x=x, linestyle=':', color='red', linewidth=1)
plt.show()


In [ ]:
import re
import matplotlib.pyplot as plt

log_path = log_file  # your full log file
target_pid = "3131681"

pid_usage = []
timestamps = []

with open(log_path, "r") as f:
    lines = f.readlines()

i = 0
step = 0
skip_next = False

while i < len(lines):
    if "NVIDIA-SMI" in lines[i]:
        if skip_next:
            skip_next = False
            i += 1
            continue

        j = i
        usage_this_block = 0
        in_process_section = False

        while j < len(lines):
            line = lines[j].strip()

            if line.startswith("| Processes:"):
                in_process_section = True

            elif in_process_section:
                if re.search(rf"\|\s+\d+\s+N/A\s+N/A\s+{target_pid}\s+", line):
                    match = re.search(r"(\d+)MiB", line)
                    if match:
                        mem = int(match.group(1))
                        usage_this_block += mem
                        print(f"Step {step}: found {mem} MiB on this GPU")

                if line.startswith("+-----------------------------------------------------------------------------------------+"):
                    break

            j += 1

        if usage_this_block > 0:
            pid_usage.append(usage_this_block)
            timestamps.append(step)
            step += 1

        i = j + 1
        skip_next = True  # skip next nvidia-smi block (the duplicate)
    else:
        i += 1

print(f"Found {len(pid_usage)} data points for PID {target_pid}")

# Plot
plt.figure(figsize=(10, 6))
plt.plot(timestamps, pid_usage, marker='o', label=f'PID {target_pid}')
plt.xlabel("nvidia-smi snapshot index")
plt.ylabel("Memory Used (MiB)")
plt.title(f"GPU Memory Usage Over Time for PID {target_pid}")
plt.grid(True)
plt.legend()
plt.tight_layout()
for x in range(0, len(timestamps)+1, 15):
    plt.axvline(x=x, linestyle=':', color='red', linewidth=1)
plt.show()


In [ ]:
gpu1_usage[:53]=[g-17246 for g in gpu1_usage[:53]]

In [ ]:
gpu1_usage

In [ ]:
# Plot
plt.figure(figsize=(10, 6))
if gpu1_usage:
    plt.plot(timestamps[:len(gpu1_usage)], gpu1_usage, label="GPU 1", marker="o")
if gpu2_usage:
    plt.plot(timestamps[:len(gpu2_usage)], gpu2_usage, label="GPU 2", marker="o")
plt.xlabel("model update step")
plt.ylabel("Memory Used (MiB)")
plt.title("GPU Memory Usage Over Time")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
import verl
print(verl.__file__)

In [ ]:
import verl.trainer
print(dir(verl.trainer))
import verl.trainer.ppo
print(dir(verl.trainer.ppo))